In [1]:
from google.colab import drive

drive.mount('/gdrive')
gdrive_root = '/gdrive/My Drive'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
batch_size = 100
 
train_dataset = datasets.MNIST(root='./mnist_data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
 
test_dataset = datasets.MNIST(root='./mnist_data/',
                             train=False,
                             transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.relu = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm2d(num_features=64)

        self.conv1 = nn.Conv2d(in_channels=1, out_channels = 128, kernel_size=3, stride = 1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=128, out_channels = 64, kernel_size=3, stride = 2, padding=2)
        self.dropout1 = nn.Dropout(0.25)
        self.FC1 = nn.Linear(64*15*15, 128)
        self.dropout2 = nn.Dropout(0.5)
        self.FC2 = nn.Linear(128, 10)
        
        
    def forward(self, x):
        in_size = x.size(0)
        x = self.relu(self.conv1(x)) 
        x = self.relu(self.conv2(x))
        x = self.dropout1(x)
        x = x.view(-1, 64*15*15)
        x = self.relu(self.FC1(x))
        x = self.dropout2(x)
        x = self.FC2(x)

        return x

In [4]:
model = Net()
 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

best_acc = 0.
ckpt_dir = os.path.join(gdrive_root, 'checkpoints')
if not os.path.exists(ckpt_dir):
  os.makedirs(ckpt_dir)

ckpt_path = os.path.join(ckpt_dir, 'modelC_ckpt.pt')
if os.path.exists(ckpt_path):
  ckpt = torch.load(ckpt_path)
  try:
    model.load_state_dict(ckpt['model'])
    optimizer.load_state_dict(ckpt['optimizer'])
    best_acc = ckpt['best_acc']
  except RuntimeError as e:
    print('wrong checkpoint')
  else:    
    print('checkpoint is loaded !')
    print('current best accuracy : %.2f' % best_acc)

checkpoint is loaded !
current best accuracy : 0.94


In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

def test(best_acc):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        
        test_loss += criterion(output, target).item()
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(test_loader.dataset)
    test_acc = correct / float(len(test_loader.dataset))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * test_acc))

    if test_acc > best_acc:
      best_acc = test_acc
      ckpt = {'model': model.state_dict(),
              'optimizer':optimizer.state_dict(),
              'best_acc':best_acc}
      torch.save(ckpt, ckpt_path)
      print('checkpoint is saved!')

In [6]:
for epoch in range(1, 10):
    train(epoch)
    test(best_acc)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.432569
Train Epoch: 1 [1000/60000 (2%)]	Loss: 0.355258
Train Epoch: 1 [2000/60000 (3%)]	Loss: 0.378677
Train Epoch: 1 [3000/60000 (5%)]	Loss: 0.493665
Train Epoch: 1 [4000/60000 (7%)]	Loss: 0.327278
Train Epoch: 1 [5000/60000 (8%)]	Loss: 0.320452
Train Epoch: 1 [6000/60000 (10%)]	Loss: 0.289864
Train Epoch: 1 [7000/60000 (12%)]	Loss: 0.276199
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.256064
Train Epoch: 1 [9000/60000 (15%)]	Loss: 0.194803
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.159934
Train Epoch: 1 [11000/60000 (18%)]	Loss: 0.377068
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.282747
Train Epoch: 1 [13000/60000 (22%)]	Loss: 0.245141
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.312004
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.313262
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.278554
Train Epoch: 1 [17000/60000 (28%)]	Loss: 0.196207
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.350030
Train Epoch: 1 [19000/60000 (32%)]	Loss: 0.200137
Train Epoch: 1 [200

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.0016, Accuracy: 9527/10000 (95%)

checkpoint is saved!
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.249352
Train Epoch: 2 [1000/60000 (2%)]	Loss: 0.218792
Train Epoch: 2 [2000/60000 (3%)]	Loss: 0.209071
Train Epoch: 2 [3000/60000 (5%)]	Loss: 0.285269
Train Epoch: 2 [4000/60000 (7%)]	Loss: 0.217480
Train Epoch: 2 [5000/60000 (8%)]	Loss: 0.197383
Train Epoch: 2 [6000/60000 (10%)]	Loss: 0.267350
Train Epoch: 2 [7000/60000 (12%)]	Loss: 0.330847
Train Epoch: 2 [8000/60000 (13%)]	Loss: 0.182075
Train Epoch: 2 [9000/60000 (15%)]	Loss: 0.229490
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.302622
Train Epoch: 2 [11000/60000 (18%)]	Loss: 0.296489
Train Epoch: 2 [12000/60000 (20%)]	Loss: 0.234368
Train Epoch: 2 [13000/60000 (22%)]	Loss: 0.220704
Train Epoch: 2 [14000/60000 (23%)]	Loss: 0.162031
Train Epoch: 2 [15000/60000 (25%)]	Loss: 0.342435
Train Epoch: 2 [16000/60000 (27%)]	Loss: 0.187945
Train Epoch: 2 [17000/60000 (28%)]	Loss: 0.227372
Train Epoch: 2 [18000/60000 (30%)]	Lo